In [1]:
import pandas as pd

df = pd.read_csv("/home/squirrel/ccg-new/projects/perspective/data/idebate/norm_persp_annotations_batch2.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18049 entries, 0 to 18048
Data columns (total 6 columns):
id                18049 non-null int64
author            18049 non-null object
claim_id          18049 non-null int64
perspective_id    18049 non-null int64
rel               18049 non-null object
comment           0 non-null float64
dtypes: float64(1), int64(3), object(2)
memory usage: 846.1+ KB


In [2]:
# Drop GOLD and TEST annotations
df = df[df.author != "GOLD"]
df.info()
df = df[df.author != "TEST"]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13913 entries, 4136 to 18048
Data columns (total 6 columns):
id                13913 non-null int64
author            13913 non-null object
claim_id          13913 non-null int64
perspective_id    13913 non-null int64
rel               13913 non-null object
comment           0 non-null float64
dtypes: float64(1), int64(3), object(2)
memory usage: 760.9+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 13879 entries, 4170 to 18048
Data columns (total 6 columns):
id                13879 non-null int64
author            13879 non-null object
claim_id          13879 non-null int64
perspective_id    13879 non-null int64
rel               13879 non-null object
comment           0 non-null float64
dtypes: float64(1), int64(3), object(2)
memory usage: 759.0+ KB


In [3]:
# Label Distributions
print(df.groupby('rel').count())

       id  author  claim_id  perspective_id  comment
rel                                                 
N    1870    1870      1870            1870        0
S    6138    6138      6138            6138        0
U    5871    5871      5871            5871        0


In [66]:
import numpy as np
def fleiss_kappa_1d(arr):
    """
    Easy version of fleiss kappa, where there is only one example
    """
    
    num_rater = np.sum(arr)
#     print(num_rater)
    p_e = np.sum(np.square(np.divide(arr, num_rater)))
#     print(p_e)
    p_o = 1/(num_rater * (num_rater - 1)) * (np.sum(np.square(arr)) - num_rater)
#     print(p_o)
    if p_e == 1: 
        return 1
    else:
        return (p_o - p_e) / (1 - p_e)


In [11]:
# IAA analysis -- Macro Avg Fleiss' Kappa among claims 
# Using NLTK implementation 
# https://www.nltk.org/_modules/nltk/metrics/agreement.html
from nltk.metrics.agreement import AnnotationTask
import numpy as np

claims = df.claim_id.unique()

not_complete_author = {
"A1JRMSNROZ1JX5",
"A2FARB7LA24AU8",
"AWZR4H0QX23W4",
"A149ROBL26JWPJ",
"AWJUGWPCUGKEG",
"A3U74S280Z4B0",
"AUCHGHY1IKZZK",
"A2SWQM5X54P1O5",
"A1OFSWG4TA4VVR",
"A1DNJ17PE2RYJZ",
"A2541C8MY0BYV3",
"A3QSFE6GKO157S",
"A95RHRMLHT89M",
"A1ZD8RU6YB0VEU",
"AU2RMH9IZP60M",
"A3C5G8LGIAW0XL",
"AVI7K876BV3QL",
"A3C70KUK27HI3Y",
"A2TBXASXZIRNNW",
"A14HW8I4RYHNC4",
"A3RTXKL0EGLRDJ",
"A52FUUEDGSZT9",
"A2YR5M2I1YCH75",
"A2TZA6QIDOHF0R",
"A3TLI95L5GOBKG",
"AROOCBM042SJD",
"A2P065E9CYMYJL",
"A2E5PZUD6W4OA1",
"A2E5PZUD6W4OA1",
"A1ZT30BGR3266K",
"A3GKCB2D8O2ICZ",
"A2C73Y1COWCA51",
"A3RYI5HXC2MJLN",
"A1YTNGH5SMM2CJ",
"ALJMVJ1L3NUTE",
"AQYYBOOF0846O",
"A2B6WQG0A9CKXQ",
"ALEJV7D94ZLHF",
"A5J0OW727ZCWY",
}

arr_al = []
_data = []

for cid in claims:
    c_df = df[df.claim_id == cid]
    c_df = c_df[~c_df.author.isin(not_complete_author)]
    
    persps = c_df.perspective_id.unique()
    
    for pid in persps:
        p_df = c_df[c_df.perspective_id == pid]
        
        group = p_df.rel.value_counts()
        
        _s = _u = _n = 0
        if "S" in group.index:
            _s = group.S
        if "U" in group.index:
            _u = group.U
        if "N" in group.index:
            _n = group.N
        
        _data.append([cid, pid, _s, _u, _n])

print(len(_data))
with open("/mnt/e/pilot_batch2_result.csv", 'w+') as fout:
    fout.write("claim,perspective,sup,und,ns\n")
    for entry in _data:
        _entry = [str(x) for x in entry]
        fout.write(",".join(_entry)+'\n')

    
# for entry in _data
    
#     triples = []
#     for idx, row in c_df.iterrows():
#         triples.append((row.author, row.perspective_id, row.rel))
    
#     an = AnnotationTask(data=triples)
#     k = an.multi_kappa()
#     arr_al.append(k)

# print("Avg IAA:{}".format(np.mean(arr_al)))
# print("Stddev:{}".format(np.std(arr_al)))

1222


In [65]:
fleiss_kappa_1d([30, 0 ,0])

30
1.0
1.0


/home/squirrel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


nan